In [1]:
import pandas as pd

In [ ]:
# treating the data
binance = pd.read_csv('data/processed/extrato_binance.csv', index_col=0)
binance.sort_index(inplace=True)
binance['UTC_Time'] = pd.to_datetime(binance['UTC_Time'])

In [ ]:
pd.set_option('display.float_format', lambda x: '%.8f' % x) 
# change the visualization of a float to eight decimal digits.

In [ ]:
# fix one register in a dumb form 
binance.loc[57,'UTC_Time'] = binance.loc[56,'UTC_Time']

In [ ]:
# Preparing data
def cache_transaction(df, column):
    df['Ticker_cache'] = df['Coin'].shift() * (df[column] == df[column].shift())
    df['Value_cache'] = df['Change'].shift() * (df[column] == df[column].shift())
    return 
cache_transaction(binance, 'UTC_Time')

In [ ]:
# Guardando info do saldo na binance
wallet = binance.groupby('Coin').agg({'UTC_Time': 'first'}).sort_values('UTC_Time') # Primeira ocorrência de cada moeda
wallet['Balance'] = 0
wallet['BRL_spent'] = 0


In [ ]:
# trezor
index_names = list(binance[binance['Operation'].isin(['Fiat Withdraw', 'Withdraw'])]['Coin'].unique())
data = {'Amount': [0.0]*len(index_names), 'BRL_spent': [0.0]*len(index_names)}
coldwallet = pd.DataFrame(data, index=index_names)

In [5]:
binance.apply(triagem, axis=1) 

ValueError: triagem can only be applied to a DataFrame once.

In [4]:
# put the decorator here

def triagem(serie):
    operation = serie['Operation']
    
    actions = {
        'Deposit': deposit,
        'Transaction Related': buy,
        'Large OTC Trading': convert,
        'Fiat Withdraw': withdraw,
        'Staking Purchase': change_amount,
        'Launchpool Interest': change_amount,
        'Simple Earn Flexible Interest': change_amount,
        'Staking Rewards': change_amount,
        'Staking Redemption': change_amount,
        'Cash Voucher Distribution': change_amount,
        'Withdraw': withdraw,
        'Small Assets Exchange BNB': multiple_BNB_trades,
        'Distribution': change_amount,
        'Simple Earn Flexible Subscription': lambda x: None,
        'Savings Distribution': lambda x: None,
        'Simple Earn Flexible Redemption': lambda x: None
    }
    
    action = actions.get(operation)
    
    if action is None:
        raise ValueError('Operation not recognized')
    
    action(serie)

       

def deposit(serie):
    if serie['Coin']=='BRL':
        wallet.loc[serie['Coin'],'Balance'] += serie['Change']
        wallet.loc[serie['Coin'],'BRL_spent'] += serie['Change']
    else:
        BRL_value = coldwallet.loc[serie['Coin'],'BRL_spent']/coldwallet.loc[serie['Coin'],'Amount']*serie['Change']
        wallet.loc[serie['Coin'],'Balance'] += serie['Change']
        wallet.loc[serie['Coin'],'BRL_spent'] += BRL_value
        coldwallet.loc[serie['Coin'],'Amount'] -= serie['Change']
        coldwallet.loc[serie['Coin'],'BRL_spent'] -= BRL_value
    

def buy(serie):

    if serie['Ticker_cache']=='': 
        return

    if serie['Coin']=='BRL': 

        if serie['Change']<0: 
            wallet.loc[serie['Ticker_cache'],'BRL_spent'] -= serie['Change'] 
        else: 
            wallet.loc[serie['Ticker_cache'],'BRL_spent'] += wallet.loc[serie['Ticker_cache'],'BRL_spent'] / wallet.loc[serie['Ticker_cache'],'Balance'] * serie['Value_cache'] 

    elif serie['Ticker_cache']=='BRL':

        if serie['Value_cache']<0: 
            wallet.loc[serie['Coin'],'BRL_spent'] -= serie['Value_cache']
        else:
            wallet.loc[serie['Coin'],'BRL_spent'] += wallet.loc[serie['Coin'],'BRL_spent'] / wallet.loc[serie['Coin'],'Balance'] * serie['Change']
        
    else:
        print('No BRL involved in this transaction')
    
    # update balance
    wallet.loc[serie['Coin'],'Balance'] += serie['Change'] 
    wallet.loc[serie['Ticker_cache'],'Balance'] += serie['Value_cache'] 
    wallet.loc['BRL', 'BRL_spent'] = wallet.loc['BRL', 'Balance']

    return

def convert(serie):

    if serie['Ticker_cache']=='': 
        return
    
    if serie['Coin'] == 'BRL' or serie['Ticker_cache'] == 'BRL':
        buy(serie)
        return
 
    if serie['Change']<0: # means that serie['Coin'] is leaving; serie['Change'] is negative
        # serie['Coin'] BRL equivalent leaving
        temp = wallet.loc[serie['Coin'],'BRL_spent'] / wallet.loc[serie['Coin'],'Balance'] * serie['Change'] # aways negative
        
    else: # means that serie['Ticker_cache'] is leaving, serie['Value_cache'] is negative
        temp = -wallet.loc[serie['Ticker_cache'],'BRL_spent'] / wallet.loc[serie['Ticker_cache'],'Balance'] * serie['Value_cache'] # temp goes positive here


    # update BRL_spent
    wallet.loc[serie['Ticker_cache'], 'BRL_spent'] -= temp # tira se Ticker_cache for saída
    wallet.loc[serie['Coin'], 'BRL_spent'] += temp # temp é negativa aqui; menos com mais - 

    # update balance
    wallet.loc[serie['Coin'],'Balance'] += serie['Change'] 
    wallet.loc[serie['Ticker_cache'],'Balance'] += serie['Value_cache'] 



def withdraw(serie):
    if serie['Coin']=='BRL': # test to avoid mean price (not needed, but saves computing time (worth it?))
        coldwallet.loc['BRL', 'Amount'] -= serie['Change'] # serie['Change'] is negative
        coldwallet.loc['BRL', 'BRL_spent'] -= serie['Change'] # eliminate this need
        wallet.loc['BRL','Balance'] += serie['Change']  # eliminate this need
        wallet.loc['BRL','BRL_spent'] += serie['Change']

    else:
        coldwallet.loc[serie['Coin'], 'Amount'] -= serie['Change']
        temp = wallet.loc[serie['Coin'],'BRL_spent'] / wallet.loc[serie['Coin'],'Balance'] * serie['Change']
        coldwallet.loc[serie['Coin'], 'BRL_spent'] -=  temp
        wallet.loc[serie['Coin'],'Balance'] += serie['Change']
        wallet.loc[serie['Coin'],'BRL_spent'] += temp


def change_amount(serie):
    '''Change the amount without alter cost '''
    wallet.loc[serie['Coin'],'Balance'] += serie['Change']


def multiple_BNB_trades(serie):
    if serie['Coin']=='BNB':
        wallet.loc['BNB', 'Balance'] += serie['Change']

    else:
        wallet.loc[serie['Coin'], 'Balance'] += serie['Change']
        wallet.loc['BNB', 'BRL_spent'] += wallet.loc[serie['Coin'], 'BRL_spent']
        wallet.loc[serie['Coin'], 'BRL_spent'] = 0

In [ ]:
consolidado = wallet[wallet['Balance']>=0.00000001].copy()
consolidado = consolidado.merge(coldwallet, left_index=True, right_index=True, how='left')
consolidado['BRL_spent_x'] += consolidado['BRL_spent_y']
consolidado['Balance'] += consolidado['Amount']
consolidado['mean_price'] = consolidado['BRL_spent_x']/consolidado['Balance']
consolidado


In [ ]:
coldwallet['mean_price'] = coldwallet['BRL_spent']/coldwallet['Amount']/5.20
coldwallet

In [3]:
# decorator que ta dando errado, retornando erro na primeira apply
def apply_once(func):
    """
    Decorator to ensure that a function is only applied to a DataFrame once.
    """
    applied = False
    def wrapper(df, *args, **kwargs):
        nonlocal applied
        if applied:
            raise ValueError(f"{func.__name__} can only be applied to a DataFrame once.")
        applied = True
        return func(df, *args, **kwargs)
    return wrapper



#@apply_once


# Visualizações dos dados

In [ ]:
# filtra as operações fora dessa lista
binance[~binance['Operation'].isin(['Deposit', 'Transaction Related', 'Large OTC Trading','Fiat Withdraw'])]


# filtra até essa data
binance[binance['UTC_Time']<'2022-04-28 13:30:52']

# Compara datas, armazena na coluna 'delta'; depois filtre pra gente detectar porblemas de pequenos deltas em buy()
binance['delta'] = binance['UTC_Time'] - binance['UTC_Time'].shift() 

# visualizar o dataframe inteiro
print(binance[binance['Ticker_cache']!=''].sort_values('Value_cache').to_markdown())

# deram esses NaNs com meu algo
lista_NaNs = ['2022-04-28 13:30:52','2022-05-08 22:11:09','2022-05-08 22:39:58','2022-05-10 11:14:31','2022-05-11 21:18:48',
'2022-05-11 21:48:38','2022-06-07 02:03:47','2022-06-09 15:42:07','2022-06-10 20:48:55','2022-06-13 11:57:29','2022-06-13 13:21:30',
'2022-06-14 02:33:07','2022-06-15 21:32:28','2022-06-21 16:56:12','2022-07-01 12:32:30','2022-07-20 12:12:42','2022-07-24 21:32:59',
'2022-11-20 01:06:39','2022-11-20 01:07:24','2022-11-20 01:41:55','2022-11-22 02:44:20','2022-11-23 12:27:00','2022-12-03 19:27:12',
'2022-12-13 13:06:25','2023-01-07 23:36:07','2023-01-16 19:19:50','2023-01-16 19:20:06','2023-01-20 21:45:55']
binance[binance['UTC_Time'].isin(lista_NaNs)]

# Somando entrada e saída; zero significa sem prejuízos (não deu zero haha)
binance[binance['Operation'].isin(['Staking Purchase', 'Staking Redemption'])].sort_values('Coin')['Change'].sum()

